In [3]:
%reload_ext autoreload
%autoreload 2

import pandas as pd

from src.paths import DATA_DIR, RAW_DATA_DIR, TEMPORARY_DATA, TRAINING_DATA, PARENT_DIR
from src.data_extraction import load_raw_data
from src.data_transformations import (
  clean_raw_data, transform_cleaned_data_into_ts_data, transform_ts_into_training_data
)

## Get the raw data

### All of 2023 

In [ ]:
trips_2023 = list(load_raw_data(year=2023))

### January 2024

In [ ]:
jan_2024 = list(
  load_raw_data(year=2024, months=[1])
)

In [ ]:
# Form a dataframe by concatenating all the available months of 2023 and January 2024
trips = pd.concat(trips_2023+jan_2024)

## Clean the data

In [ ]:
# Clean the data
clean_trips = clean_raw_data(trips)

In [ ]:
clean_trips

## Define datasets

In [ ]:
starts = clean_trips[
    ["start_time", "start_latitude", "start_longitude"]
]

stops = clean_trips[
    ["stop_time", "stop_latitude", "stop_longitude"]
]

## Transform datasets into time series

In [ ]:
agg_starts, agg_stops = transform_cleaned_data_into_ts_data(start_df = starts, stop_df = stops)

### Save time series

In [ ]:
trimmed_agg_starts = agg_starts.iloc[:,:3]
trimmed_agg_stops = agg_stops.iloc[:,:3]

trimmed_agg_starts.to_parquet(TEMPORARY_DATA/"agg_starts.parquet")
trimmed_agg_stops.to_parquet(TEMPORARY_DATA/"agg_stops.parquet")

## Transform time series data into training data

###### I am going to consider only the data from March 2023 till the end of January 2024, due to memory limitations that were encountered when attempting hyperparameter tuning on the full start dataset.


###### For now, I will focus on the data from March 2023 to October 2023. This will be converted into training data first. On the subsequent notebook, the remaining months' training data will be included by concatenation

In [ ]:
from datetime import datetime

trimmed_agg_starts = trimmed_agg_starts[
  trimmed_agg_starts["start_hour"] > datetime(2023,3,1) 
].reset_index(drop=True)

trimmed_agg_starts = trimmed_agg_starts[
  trimmed_agg_starts["start_hour"] < datetime(2023,10,30)
].reset_index(drop=True)

### Start data

In [ ]:
start_features, start_target = transform_ts_into_training_data(
    ts_data=trimmed_agg_starts,
    start_or_stop="start",
    input_seq_len=24*28*1,                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
    step_size=24
)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

print(f"{start_features.shape=}")
print(f"{start_target.shape=}")

In [ ]:
start_features["trips_next_hour"] = start_target
start_table = start_features

In [ ]:
start_table.to_parquet(TRAINING_DATA/"march_oct_start_table.parquet")

#### Saving the final tabular dataset

In [ ]:
start_features["trips_next_hour"] = start_target
start_table = start_features

In [ ]:
start_table.to_parquet(TRAINING_DATA/"start_table.parquet")

### Stop data

In [ ]:
trimmed_agg_stops = pd.read_parquet(TEMPORARY_DATA/"agg_stops.parquet")

In [5]:
stop_features, stop_target = transform_ts_into_training_data(
    ts_data=trimmed_agg_stops,
    start_or_stop="stop",
    input_seq_len=24*28*1, 
    step_size=24
)
print(f"{stop_features.shape=}")
print(f"{stop_target.shape=}")                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

100%|██████████| 1264/1264 [06:10<00:00,  3.41it/s]


stop_features.shape=(466416, 674)
stop_target.shape=(466416,)


#### Saving the final tabular dataset

In [6]:
stop_features["trips_next_hour"] = stop_target
stop_table = stop_features


stop_table.to_parquet(TRAINING_DATA/"jan_to_jan_stops.parquet")